In [ ]:
%%capture
!pip install transformers
!pip install opendatasets --upgrade
!pip install pywer

In [ ]:
from transformers import (
    EarlyStoppingCallback,
    Trainer,
    TrainingArguments,
    Wav2Vec2CTCTokenizer,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2ForCTC,
    Wav2Vec2Processor,
    trainer_utils
)

In [ ]:
import torch
import torchaudio
import numpy as np
import pandas as pd

Download Test Dataset from Kaggle


In [ ]:
import opendatasets as od
dataset_url = 'https://www.kaggle.com/datasets/aashish12321/nepali-tts-test-dataset-cleaned'
od.download(dataset_url)

100%|██████████| 1.59G/1.59G [01:19<00:00, 21.6MB/s]


In [ ]:
colnames=['path','labels']
audio_path='/content/nepali-tts-test-dataset-cleaned'
transcript_path='/content/nepali-tts-test-dataset-cleaned/final.csv'
df=pd.read_csv(transcript_path,names=colnames)
df['path']=audio_path+df['path']
print(df.shape)
df.head()

(2938, 2)


,path,labels
0,/content/nepali-tts-test-dataset-cleanedpath,labels
1,/content/nepali-tts-test-dataset-cleaned/audio...,त्यसै गरी विश्वको कुनाकुनामा शान्तिको संदेश छर...
2,/content/nepali-tts-test-dataset-cleaned/audio...,श्वेतभैरबी केवल बुट्टाका अस्पष्ट रेखाहरू मोटो ...
3,/content/nepali-tts-test-dataset-cleaned/audio...,हाम्रो देश स्वर्ग झैँ सुन्दर र चद्रमा झैँ चम्क...
4,/content/nepali-tts-test-dataset-cleaned/audio...,"ढोकाको डन्डीको छाया एउटा भुइँमा सोझो, अर्को भि..."


In [ ]:
df.drop(0,inplace=True)

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("Ashwini1412/wav2vec2-nepali-final").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("Ashwini1412/wav2vec2-nepali-final")

config.json:   0%|          | 0.00/2.31k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/814 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def segmentLargeArray(inputTensor,chunksize=200000):
    # print(inputTensor)
    list_of_segments = []
    tensor_length = inputTensor.shape[1]
    for i in range(0,tensor_length+1,chunksize):
        list_of_segments.append(inputTensor[:,i:i+chunksize])
    return list_of_segments

In [ ]:
def predict_from_speech(file):
    speech_array, sampling_rate = torchaudio.load(file)
    # print(speech_array,sampling_rate)
    resampler = torchaudio.transforms.Resample(sampling_rate, 16000)
    resampled_array = resampler(speech_array).squeeze()
    if len(resampled_array.shape) == 1:
        resampled_array = resampled_array.reshape([1,resampled_array.shape[0]])
    # print(resampled_array.shape[1])
    if resampled_array.shape[1] >= 200000:
        # print('The input file is longer than 10 seconds')
        list_of_segments = segmentLargeArray(resampled_array)
        # print(list_of_segments)
        output = ''
        for segment in list_of_segments:
            logits = model(segment.to("cuda")).logits
            pred_ids = torch.argmax(logits,dim=-1)[0]
            output += processor.decode(pred_ids)
        # print(f"Prediction:\n{output}")
        return output
    else:
        # print('The input file is less than 10 seconds')
        logits = model(resampled_array.to("cuda")).logits
        # print(logits)
        pred_ids = torch.argmax(logits, dim = -1)[0]
        # print("Prediction:")
        output=processor.decode(pred_ids)
        return output

In [ ]:
import random
random_number=int(random.uniform(0, df.shape[0]))
test=df.loc[random_number,'path']
print(test)
predict_from_speech(test)

/content/nepali-tts-test-dataset-cleaned/audio/Voice511.wav


'सिचाीको लागि पनि वर्षा हुनुजरोरी छ'

In [ ]:
df.head()

,path,labels
1,/content/nepali-tts-test-dataset-cleaned/audio...,त्यसै गरी विश्वको कुनाकुनामा शान्तिको संदेश छर...
2,/content/nepali-tts-test-dataset-cleaned/audio...,श्वेतभैरबी केवल बुट्टाका अस्पष्ट रेखाहरू मोटो ...
3,/content/nepali-tts-test-dataset-cleaned/audio...,हाम्रो देश स्वर्ग झैँ सुन्दर र चद्रमा झैँ चम्क...
4,/content/nepali-tts-test-dataset-cleaned/audio...,"ढोकाको डन्डीको छाया एउटा भुइँमा सोझो, अर्को भि..."
5,/content/nepali-tts-test-dataset-cleaned/audio...,घरघरमा बत्ती निभेका छन्‌ ।


In [ ]:
df['predictions']= df['path'].apply(lambda x: predict_from_speech(x))

In [ ]:
df.head(30)

,path,labels,predictions
1,/content/nepali-tts-test-dataset-cleaned/audio...,त्यसै गरी विश्वको कुनाकुनामा शान्तिको संदेश छर...,त्यसैगरी विश्व को कुना कुना मा शान्तिको शन्देश...
2,/content/nepali-tts-test-dataset-cleaned/audio...,श्वेतभैरबी केवल बुट्टाका अस्पष्ट रेखाहरू मोटो ...,स्वेप भैरवी केबल बुट्टा का अस्पष्ट रेखाहरू मोट...
3,/content/nepali-tts-test-dataset-cleaned/audio...,हाम्रो देश स्वर्ग झैँ सुन्दर र चद्रमा झैँ चम्क...,हाम्रो देश स्वर्ग झैँ सुन्दर र चन्द्रमा झैँ चम...
4,/content/nepali-tts-test-dataset-cleaned/audio...,"ढोकाको डन्डीको छाया एउटा भुइँमा सोझो, अर्को भि...",डोकोको डन्डीको छायाँ एउटा भुिमा सोझ हो अर्को भ...
5,/content/nepali-tts-test-dataset-cleaned/audio...,घरघरमा बत्ती निभेका छन्‌ ।,घरघरमा बत्िनि भेका छन्
6,/content/nepali-tts-test-dataset-cleaned/audio...,यो सारा नेपालीहरूले आफ्ना मनमा लागेका कुराहरू ...,यो सारा नेपालीहरूले आफ्ना मन्मा लागेका कुराआहर...
7,/content/nepali-tts-test-dataset-cleaned/audio...,प्रारम्भमा ग्रामीण जीवनको बाटोमा तै-तै गरेर पा...,प्रारम्भमा ग्रामीण जीवनको बाटोमा तै तै गरेर पा...
8,/content/nepali-tts-test-dataset-cleaned/audio...,"लागि सञ्चारका माध्यमले अहम् भूमिका खेलेको छ, प...",लागि सञ्चारका माध्यमले अहमखुमिका खेलेको छ पत्र...
9,/content/nepali-tts-test-dataset-cleaned/audio...,जस्तै –हाम्रो सहर हेटौँडा ।,जस्तै हाम्रो सहर हेटौडा
10,/content/nepali-tts-test-dataset-cleaned/audio...,पहिले जो आमाजस्ती भएर मेरो औंलालाई सहारा दिँदै...,पहिले जो आमा जस्ती भएर मेर औँला लाई साहाँरा दि...


In [ ]:
df.shape

(2937, 3)

In [ ]:
df.to_csv('testing.csv', index=False)

In [ ]:
predictions=df['predictions'].tolist()
labels=df['labels'].tolist()

In [ ]:
import pywer

In [ ]:
wer=pywer.wer(predictions,labels)
cer=pywer.cer(predictions,labels)
print(f"Accuracy: {(100-cer):.2f}% \nWER: {wer:.2f}% \nCER: {cer:.2f}%")

Accuracy: 78.54% 
WER: 64.86% 
CER: 21.46%


40.155595451825256 9.138913841013457


In [ ]:
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 81.8 MB/s eta 0:00:00


In [ ]:
import Levenshtein

def calculate_wer(reference, hypothesis):
    reference_words = reference.split()
    hypothesis_words = hypothesis.split()

    distance = Levenshtein.distance(reference_words, hypothesis_words)

    wer = distance / len(reference_words)
    return wer

def calculate_cer(reference, hypothesis):
    reference_chars = list(reference)
    hypothesis_chars = list(hypothesis)

    distance = Levenshtein.distance(reference_chars, hypothesis_chars)

    cer = distance / len(reference_chars)
    return cer


In [ ]:
predicted_text=df.loc[1,'predictions']
label=df.loc[1,'labels']

In [ ]:
x=calculate_wer(predicted_text,label)
y=calculate_cer(label,predicted_text)
print(x,y)

0.6086956521739131 0.11570247933884298


In [ ]:
print(df.shape[0])

2937


In [ ]:
total_cer=0
total_wer=0

for i in range(1,df.shape[0]):
    predicted_text=df.loc[i,'predictions']
    label=df.loc[i,'labels']
    wer=calculate_wer(label,predicted_text)
    cer=calculate_cer(label,predicted_text)
    total_cer+=cer
    total_wer+=wer

print(total_cer/df.shape[0],total_wer/df.shape[0])


0.1236273288837004 0.46229638899008746


In [ ]:
accuracy = (1 - total_cer/df.shape[0])*100
wer=(total_wer/df.shape[0]) *100
cer=(total_cer/df.shape[0]) *100
print(f"Accuracy: {accuracy:.2f}% \nWER: {wer:.2f}% \nCER: {cer:.2f}%")

Accuracy: 87.64% 
WER: 46.23% 
CER: 12.36%
